In [21]:
import mytools.regression as reg
import numpy as np
import matplotlib.pyplot as plt
import datetime

## Sigmoid fitting for intensive care patients

In [22]:
data = np.genfromtxt(fname='italy-intensive_care.csv', delimiter=',', names=True)

x_orig = data['day']
y_orig = data['intensive_care']

model, xp, pxp = reg.fit_sigmoid(x_orig, y_orig)

flex = reg.sigmoid_get_flex(model)

# Plot the results
plt.plot(x_orig, y_orig, '.', label='Intensive care cases')
plt.plot(xp, pxp, '-', label='fitting sigmoid')
plt.plot(flex[0], flex[1], '.', label='Inflection point ('+reg.day_of_year_to_date(flex[0]).strftime("%d %b")+' '+'{:.2f}'.format(flex[1])+' cases)')
locs, labels = plt.xticks()
a = list((reg.day_of_year_to_date(v)).strftime("%d %b") for v in locs.tolist())
plt.xticks(ticks=locs.tolist(), labels=a)

plt.ylabel('cases', rotation='horizontal')
plt.grid(True)
plt.title('Italy - Intensive care patients')
plt.legend()
plt.show()


AttributeError: module 'mytools.regression' has no attribute 'sigmoid_get_flex'